In [1]:
import json
import pandas as pd
from functions_and_methods import get_opsea_trans_by_collec, combine_json_files, get_traits_collection, get_nft_traits, calculate_rarity_scores, add_rarity_score

### 0. Parameter Specification

In [2]:
data = "boredapeyachtclub, mutant-ape-yacht-club, bored-ape-kennel-club"

words = data.split(',')  # Returns a list
collection_list = ['' + word.strip() + '' for word in words]  # I split it at , and then append to the list
after = '2020-01-01 01:00:00'  # 1st of January 2020 01:00:00 am
before = '2023-12-01 01:00:00'  # 1st of December 2023 01:00:00 am
event_type = 'sale'  # Other types: all, cancel, redemption, transfer

### 1. Downloading transactions

In [ ]:
for collection in collection_list:
    get_opsea_trans_by_collec(collection, before, after, event_type, collection)

### 2. Combine the JSON files fo transactions into one

In [ ]:
# Combine the JSON files into one
combine_json_files(collection_list, 'combined_transactions')

### 3. Add to a dataframe

In [4]:
with open('Transaction_files/combined_transactions.json') as z:
    trans_3years = json.load(z)

In [6]:
df = pd.json_normalize(trans_3years)
df.head()  # 28 columns

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol,nft,payment
0,sale,,ethereum,,1701378215,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xae7fca4f38a28fe3580d42d7a56190ff4b80dc65,0xb085fc8e1fa7518ced3c411a9184551138db03c75a53...,1701378215,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-13T02:50:25.797011,False,False,28250000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
1,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x46667400bc0225903e71d4f16f1625b239c72617a326...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2023-12-27T06:22:13.050808,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
2,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x9afef7dac35f070dc3976f0597b8c872f6b19d1d,0x998eac1b78861ce9ba700003b21582f3d31d4d9b12e1...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2023-12-02T07:21:27.507469,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
3,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x50f53f1b2900bbdb10fa54e29817eb0a12f21739d8c3...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-26T08:08:15.693777,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
4,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x157175e7dee4ed8269415be4cf7c52fbbaa3eb113656...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-26T17:09:27.131954,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN


### 4. Number of unique NFTs

In [8]:
df["nft.metadata_url"].nunique(), df["nft.identifier"].nunique() # Only urls are truly unique 30860 is the correct number

(30860, 15644)

### 5. Extracting traits from metadata_url for every transaction

In [ ]:
# Get DataFrame with added nft.traits column
df = get_nft_traits(df)

In [ ]:
df.to_json('Transactions_with_traits/combined_transactions_traits.json', orient='records')

In [9]:
with open('Transactions_with_traits/df_bakc_traits.json') as a:
    bakc_t = json.load(a)

In [11]:
bakc_t[0]

{'event_type': 'sale',
 'order_hash': '',
 'chain': 'ethereum',
 'protocol_address': '',
 'closing_date': 1701388595,
 'quantity': 1,
 'seller': '0x535e7039c7a7bb3afec86a6257c21b29700ae059',
 'buyer': '0x29469395eaf6f95920e59f858042f0e28d98a20b',
 'transaction': '0x592896b5a2474a9239ce73ad6a091dbe12e9c764b05a78b7c2f75ddd95c5439a',
 'event_timestamp': 1701388595,
 'nft.identifier': '8706',
 'nft.collection': 'bored-ape-kennel-club',
 'nft.contract': '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623',
 'nft.token_standard': 'erc721',
 'nft.name': None,
 'nft.description': None,
 'nft.image_url': 'https://ipfs.io/ipfs/QmawLSmZz3xjafo2RqV9mzbxM6pD4e69oqiWSu6wDaSAZL',
 'nft.metadata_url': 'https://ipfs.io/ipfs/QmTDcCdt3yb6mZitzWBmQr65AW6Wska295Dg9nbEYpSUDR/8706',
 'nft.opensea_url': 'https://opensea.io/assets/ethereum/0xba30e5f9bb24caa003e9f2f0497ad287fdf95623/8706',
 'nft.updated_at': '2023-12-03T23:10:02.002232',
 'nft.is_disabled': False,
 'nft.is_nsfw': False,
 'payment.quantity': '1630000000

In [12]:
# extracting traits from a list of dictionaries 
for item in bakc_t:
    if 'nft.traits' in item and item['nft.traits']:
        item['nft.traits'] = [trait for sublist in item['nft.traits'] for trait in sublist]

In [13]:
bakc_t[0]

{'event_type': 'sale',
 'order_hash': '',
 'chain': 'ethereum',
 'protocol_address': '',
 'closing_date': 1701388595,
 'quantity': 1,
 'seller': '0x535e7039c7a7bb3afec86a6257c21b29700ae059',
 'buyer': '0x29469395eaf6f95920e59f858042f0e28d98a20b',
 'transaction': '0x592896b5a2474a9239ce73ad6a091dbe12e9c764b05a78b7c2f75ddd95c5439a',
 'event_timestamp': 1701388595,
 'nft.identifier': '8706',
 'nft.collection': 'bored-ape-kennel-club',
 'nft.contract': '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623',
 'nft.token_standard': 'erc721',
 'nft.name': None,
 'nft.description': None,
 'nft.image_url': 'https://ipfs.io/ipfs/QmawLSmZz3xjafo2RqV9mzbxM6pD4e69oqiWSu6wDaSAZL',
 'nft.metadata_url': 'https://ipfs.io/ipfs/QmTDcCdt3yb6mZitzWBmQr65AW6Wska295Dg9nbEYpSUDR/8706',
 'nft.opensea_url': 'https://opensea.io/assets/ethereum/0xba30e5f9bb24caa003e9f2f0497ad287fdf95623/8706',
 'nft.updated_at': '2023-12-03T23:10:02.002232',
 'nft.is_disabled': False,
 'nft.is_nsfw': False,
 'payment.quantity': '1630000000

### 6. Getting traits statistics for a collection

In [3]:
for collection in collection_list:
    get_traits_collection(collection)

In [4]:
with open('Collection_traits/bored-ape-kennel-club_traits.json') as z:
    traits_bakc = json.load(z)

In [5]:
traits_bakc.keys()

dict_keys(['Background', 'Fur', 'Neck', 'Mouth', 'Eyes', 'Head', 'Feet', 'Back'])

### 7. Calculating trait scores

In [7]:
# Example usage:
total_nfts_bakc = 10000 # 9602 minted
rarity_scores_bakc = calculate_rarity_scores(traits_bakc, total_nfts_bakc)

In [8]:
rarity_scores_bakc

{'Background': {'Outhouse': 4.144218814753419,
  'SS Monke Biz': 4.278990158322636,
  'Swamp': 4.122011541632316,
  'Club Exterior': 6.097560975609756,
  'Bar': 12.72264631043257},
 'Fur': {'Radioactive': 53.191489361702125,
  'Brown': 6.570302233902759,
  'Tan': 4.301075268817204,
  'Robot': 26.109660574412533,
  'Black': 6.377551020408164,
  'Zombie': 25.641025641025642,
  'Noise': 55.24861878453038,
  'Pink': 9.920634920634921,
  'Death Bot': 38.314176245210724,
  'Galaxy': 45.24886877828054,
  'Mega Death Bot': 58.47953216374269,
  'Cheetah': 16.366612111292962,
  'Cyberpunk': 36.90036900369004,
  'Trippy': 90.90909090909092,
  'DMT': 34.013605442176875,
  'Solid Gold': 102.04081632653062},
 'Neck': {'Japan Bandana': 90.90909090909092,
  'Christmas Wreath': 55.24861878453038,
  'Plaid Bowtie': 50.76142131979696,
  'BAYC Collar': 11.025358324145534,
  'Glow Necklace': 59.523809523809526,
  'Red BAYC Bandana': 181.81818181818184,
  'Whiskey Barrel': 119.04761904761905,
  'Camo Bandan

### 8. Calculating overall rairty score

In [ ]:
add_rarity_score(bakc_t, rarity_scores_bakc)